In [0]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]


transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(),
      transforms.Normalize(mean, std)])

test_transform = transforms.Compose([transforms.ToTensor(), 
                                     transforms.Normalize(mean, std)])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=16,
                                          drop_last=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=16,
                                         drop_last=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print('\nIl y a', len(trainloader.dataset), "données d'entrainement")
print('Il y a', len(testloader.dataset), "données de teste")


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Il y a 50000 données d'entrainement
Il y a 10000 données de teste


In [0]:
class ResNeXtBottleneck(nn.Module):
    """
    RexNeXt bottleneck type C (https://github.com/facebookresearch/ResNeXt/blob/master/models/resnext.lua)
    """

    def __init__(self, in_channels, out_channels, stride, cardinality, base_width, widen_factor):
        """ Constructor
        Args:
            in_channels: input channel dimensionality
            out_channels: output channel dimensionality
            stride: conv stride. Replaces pooling layer.
            cardinality: num of convolution groups.
            base_width: base number of channels in each group.
            widen_factor: factor to reduce the input dimensionality before convolution.
        """
        super(ResNeXtBottleneck, self).__init__()
        width_ratio = out_channels / (widen_factor * 64.)
        D = cardinality * int(base_width * width_ratio)
        self.conv_reduce = nn.Conv2d(in_channels, D, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn_reduce = nn.BatchNorm2d(D)
        self.conv_conv = nn.Conv2d(D, D, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn = nn.BatchNorm2d(D)
        self.conv_expand = nn.Conv2d(D, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn_expand = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut.add_module('shortcut_conv',
                                     nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0,
                                               bias=False))
            self.shortcut.add_module('shortcut_bn', nn.BatchNorm2d(out_channels))

    def forward(self, x):
        bottleneck = self.conv_reduce.forward(x)
        bottleneck = F.relu(self.bn_reduce.forward(bottleneck), inplace=True)
        bottleneck = self.conv_conv.forward(bottleneck)
        bottleneck = F.relu(self.bn.forward(bottleneck), inplace=True)
        bottleneck = self.conv_expand.forward(bottleneck)
        bottleneck = self.bn_expand.forward(bottleneck)
        residual = self.shortcut.forward(x)
        return F.relu(residual + bottleneck, inplace=True)


class CifarResNeXt(nn.Module):
    """
    ResNext optimized for the Cifar dataset, as specified in
    https://arxiv.org/pdf/1611.05431.pdf
    """

    def __init__(self, cardinality, depth, nlabels, base_width, widen_factor=4):
        """ Constructor
        Args:
            cardinality: number of convolution groups.
            depth: number of layers.
            nlabels: number of classes
            base_width: base number of channels in each group.
            widen_factor: factor to adjust the channel dimensionality
        """
        super(CifarResNeXt, self).__init__()
        self.cardinality = cardinality
        self.depth = depth
        self.block_depth = (self.depth - 2) // 9
        self.base_width = base_width
        self.widen_factor = widen_factor
        self.nlabels = nlabels
        self.output_size = 64
        self.stages = [64, 64 * self.widen_factor, 128 * self.widen_factor, 256 * self.widen_factor]

        self.conv_1_3x3 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn_1 = nn.BatchNorm2d(64)
        self.stage_1 = self.block('stage_1', self.stages[0], self.stages[1], 1)
        self.stage_2 = self.block('stage_2', self.stages[1], self.stages[2], 2)
        self.stage_3 = self.block('stage_3', self.stages[2], self.stages[3], 2)
        self.classifier = nn.Linear(self.stages[3], nlabels)
        init.kaiming_normal(self.classifier.weight)

        for key in self.state_dict():
            if key.split('.')[-1] == 'weight':
                if 'conv' in key:
                    init.kaiming_normal(self.state_dict()[key], mode='fan_out')
                if 'bn' in key:
                    self.state_dict()[key][...] = 1
            elif key.split('.')[-1] == 'bias':
                self.state_dict()[key][...] = 0

    def block(self, name, in_channels, out_channels, pool_stride=2):
        """ Stack n bottleneck modules where n is inferred from the depth of the network.
        Args:
            name: string name of the current block.
            in_channels: number of input channels
            out_channels: number of output channels
            pool_stride: factor to reduce the spatial dimensionality in the first bottleneck of the block.
        Returns: a Module consisting of n sequential bottlenecks.
        """
        block = nn.Sequential()
        for bottleneck in range(self.block_depth):
            name_ = '%s_bottleneck_%d' % (name, bottleneck)
            if bottleneck == 0:
                block.add_module(name_, ResNeXtBottleneck(in_channels, out_channels, pool_stride, self.cardinality,
                                                          self.base_width, self.widen_factor))
            else:
                block.add_module(name_,
                                 ResNeXtBottleneck(out_channels, out_channels, 1, self.cardinality, self.base_width,
                                                   self.widen_factor))
        return block

    def forward(self, x):
        x = self.conv_1_3x3.forward(x)
        x = F.relu(self.bn_1.forward(x), inplace=True)
        x = self.stage_1.forward(x)
        x = self.stage_2.forward(x)
        x = self.stage_3.forward(x)
        x = F.avg_pool2d(x, 8, 1)
        x = x.view(-1, self.stages[3])
        return self.classifier(x)

In [4]:
ResNet = CifarResNeXt(8, 20, 10, 64, 4)
ResNet.to(device)

learning_rate = 0.05
momentum = 0.9
decay = 0.0005

optimizer = torch.optim.SGD(ResNet.parameters(), learning_rate, momentum,
                            weight_decay=decay, nesterov=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


In [0]:
# ResNet.load_state_dict(torch.load('ResNet_train_10.cpkt'))

<All keys matched successfully>

In [0]:
criterion = nn.CrossEntropyLoss()
epochs = 30

c1_train_loss = []
c1_test_loss = []
c1_train_prec = []
c1_test_prec = []
c1_mean_train_loss = []
c1_mean_test_loss = []

best_state_dict = ([], 0.0)

nb_train = len(trainloader) * 256
nb_test = len(testloader) * 256

In [0]:
print("Training Begin")
for ep in range(epochs):

    sum_train = 0
    sum_test = 0
    c1_train_loss = []
    c1_test_loss = []
    print("\n\nepoch:", ep)
    ResNet.train()

    print('|         |')
    for i, (x, y) in enumerate(trainloader):
        if i % 20 == 0: print('X', end='')
        y = y.to(device)
        output = ResNet(x.to(device))

        optimizer.zero_grad()
        loss = criterion(output, y)
        
        c1_train_loss.append(float(loss))
        sum_train += int(torch.sum(torch.argmax(output, 1) == y))

        loss.backward()
        optimizer.step()
        break
    print('X')
         
    c1_mean_train_loss.append(sum(c1_train_loss)/len(c1_train_loss))
    print("train : loss =", round(c1_mean_train_loss[-1],6), 'precision =' ,round(sum_train/nb_train, 3))
    c1_train_prec.append(sum_train/nb_train)

    ResNet.eval()

    print('|         |')
    for i,(x,y) in enumerate(testloader):
        if i % 4 == 0: print('X', end='')
        with torch.no_grad():
            y = y.to(device)
            output = ResNet(x.to(device))
            loss = criterion(output, y)
            
            # accuracy
            c1_test_loss.append(float(loss))
            sum_test += int(torch.sum(torch.argmax(output,1)==y))
            break
    print('X')


    
    c1_mean_test_loss.append(sum(c1_test_loss)/len(c1_test_loss))
    print("test : loss =", round(c1_mean_test_loss[-1],6), 'precision =' ,round(sum_test/nb_test, 3))
    c1_test_prec.append(sum_test/nb_test)
    

    if c1_test_prec[-1] > best_state_dict[1]:
        print('state_dict changé')
        best_state_dict = (ResNet.state_dict(), c1_test_prec[-1])

    if len(c1_mean_test_loss) > 3 and c1_mean_test_loss[-1] > sum(c1_mean_test_loss[-3:-1])/2:
        print("End training")
        torch.save(best_state_dict[0], 'ResNet_train_final.cpkt')
        print('ResNet save, ResNet_train_final.cpkt')
        break